In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics

from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import QuantileTransformer
from sklearn.preprocessing import MaxAbsScaler


from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier

import numbers
import datetime
from sklearn import model_selection

In [2]:
# Loading data from bitcoin, gold and nasdaq
data_for_use = pd.read_csv('./bitcoin_data_for_use_v4.csv', index_col=0)
data_gold = pd.read_csv('./gold_data_for_use.csv', index_col=0)
data_nasdaq = pd.read_csv('./nasdaq_data_for_use.csv', index_col=0)

#Trial after trial I've seen that the columns "day" and "outlier" add no forecasting capacity
data_for_use_w_vol= data_for_use.drop(labels=['dia', 'outlier'], axis=1)

data_for_use_w_gold = data_for_use_w_vol.merge(data_gold, left_index=True, right_index=True)
data_for_use_w_nasdaq = data_for_use_w_vol.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_w_all = data_for_use_w_gold.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_basic = data_for_use.drop(labels=['dia',
                    'varV0','varV1','varV2','varV3','varV4','varV5','varV6','varV7','varV8','varV9',
                    'varV10','varV11','varV12','varV13','varV14','varV15','varV16','varV17','varV18','varV19',
                    'varV20','varV21','varV22','varV23','varV24','varV25','varV26','varV27','varV28','varV29', 'outlier'
                    ], axis=1)

data_for_use_basic_w_gold = data_for_use_basic.merge(data_gold, left_index=True, right_index=True)
data_for_use_basic_w_nasdaq = data_for_use_basic.merge(data_nasdaq, left_index=True, right_index=True)
data_for_use_basic_w_all = data_for_use_basic_w_gold.merge(data_nasdaq, left_index=True, right_index=True)

data_for_use_bone_deep = data_for_use_basic.drop(labels=['varPm93',
                    'varPm123','varPm153','varPm183','varPm213','varPm243','varPm273','varPm303','varPm333',
                    'varPs30','varPs37','varPs44','varPs51','varPs58','varPs65','varPs72','varPs79','varPs86'
                    ], axis=1)

data = []
data.append(['data_for_use_w_vol', data_for_use_w_vol])
data.append(['data_for_use_w_gold', data_for_use_w_gold])
data.append(['data_for_use_w_nasdaq', data_for_use_w_nasdaq])
data.append(['data_for_use_w_all', data_for_use_w_all])
data.append(['data_for_use_basic', data_for_use_basic])
data.append(['data_for_use_basic_w_gold', data_for_use_basic_w_gold])
data.append(['data_for_use_basic_w_nasdaq', data_for_use_basic_w_nasdaq])
data.append(['data_for_use_basic_w_all', data_for_use_basic_w_all])
data.append(['data_for_use_bone_deep', data_for_use_bone_deep])

In [3]:
modelos=[]

#LogisticRegression and SVC models have never offered a good prediction during trials.
#In order to save time, I don't execute them
#I'm also reducing the executions of KNeighborsClassifier to 20 neighbors because in trials there's never been
#good results from 20 to 30
#I reduce then max depth to 20 in DecisionTreeClassifier because with any number higher than thar the model overfits

#modelos.append(['LogisticRegression_none_newton-cg', LogisticRegression(penalty='none', solver='newton-cg', random_state=0)])
#modelos.append(['LogisticRegression_none_lbfgs', LogisticRegression(penalty='none', solver='lbfgs', random_state=0)])
#modelos.append(['LogisticRegression_none_sag', LogisticRegression(penalty='none', solver='sag', random_state=0)])
#modelos.append(['LogisticRegression_none_saga', LogisticRegression(penalty='none', solver='saga', random_state=0)])
#modelos.append(['LogisticRegression_l2_newton-cg', LogisticRegression(penalty='l2', solver='newton-cg', random_state=0)])
#modelos.append(['LogisticRegression_l2_lbfgs', LogisticRegression(penalty='l2', solver='lbfgs', random_state=0)])
#modelos.append(['LogisticRegression_l2_liblinear', LogisticRegression(penalty='l2', solver='liblinear', random_state=0)])
#modelos.append(['LogisticRegression_l2_sag', LogisticRegression(penalty='l2', solver='sag', random_state=0)])
#modelos.append(['LogisticRegression_l2_saga', LogisticRegression(penalty='l2', solver='saga', random_state=0)])
#modelos.append(['LogisticRegression_l1_liblinear', LogisticRegression(penalty='l1', solver='liblinear', random_state=0)])
#modelos.append(['LogisticRegression_l1_saga', LogisticRegression(penalty='l1', solver='saga', random_state=0)])
#modelos.append(['LogisticRegression_elasticnet_saga', LogisticRegression(penalty='elasticnet', l1_ratio=0.5, solver='saga', random_state=0)])
for vecinos in range(1,21):
    modelos.append(['KNeighbors_'+str(vecinos), KNeighborsClassifier(n_neighbors=vecinos)])
#for grado in range(1,4):
#    modelos.append(['SVC_'+str(grado), SVC(probability=True, kernel='poly', degree=grado, random_state=0)])
for profund in range(1,21):
    modelos.append(['DecisionTree_'+str(profund), DecisionTreeClassifier(max_depth=profund, random_state=0)])
for estimadores in range(1,21):
    modelos.append(['AdaBoost_'+str(estimadores), AdaBoostClassifier(n_estimators=estimadores, random_state=0)])
for min_muestras in range(2,21):
    modelos.append(['RandomForest_'+str(min_muestras), RandomForestClassifier(min_samples_split=min_muestras, random_state=0)])

In [4]:
def walk_forward_validation (model, X_train_wfv, y_train_wfv, X_test_wfv, y_test_wfv, real_test_wfv, ct_wfv):
    y_pred_wfv = list()
    #variable que indica si, siguiendo el modelo, estaríamos invertidos o no
    dentro=True
    resultado_naive=1
    resultado_mod=1
    resultado_mod_comis=1
    
    
    for i in range(len(y_test_wfv)):
        X_train_wfv_ct=ct_wfv.fit_transform(X_train_wfv)
        model.fit(X_train_wfv_ct, y_train_wfv)
        X_test_wfv_ct=ct_wfv.transform(X_test_wfv)
        y_pred_next = model.predict(X_test_wfv_ct[i:i+1])
        y_pred_wfv.append(y_pred_next[0])
        X_train_wfv=X_train_wfv.append(X_test_wfv[i:i+1])
        y_train_wfv=y_train_wfv.append(pd.Series(y_test_wfv[i]))
        
        
        #suponemos una comisión por compra/venta del 0,15%
        if y_pred_next[0]!=dentro:
            resultado_mod_comis=resultado_mod_comis*(1-0.0015)
        dentro=y_pred_next[0]
        
        #el naive seimpre se mantiene comprado (o siempre supone que sube)
        resultado_naive=resultado_naive*(1+real_test_wfv[i])
        #si estamos comprados entonces se nos aplica la subida o bajada del día siguiente
        if dentro:
            resultado_mod=resultado_mod*(1+real_test_wfv[i])
            resultado_mod_comis=resultado_mod_comis*(1+real_test_wfv[i])
    
    return metrics.confusion_matrix(y_test_wfv, y_pred_wfv), metrics.f1_score(y_test_wfv,y_pred_wfv), \
        metrics.accuracy_score(y_test_wfv,y_pred_wfv), resultado_naive, resultado_mod, resultado_mod_comis

In [18]:
resultados=[]

#bucle de datos
for d in data:
    data_wfv = d[1]
    #data_train, data_test = train_test_split(data_wfv, shuffle=False, test_size=0.01)
    
    #I split the data into test and the rest (test data is not meant to be used here, this program is only for CV)
    data_no_test, data_test = train_test_split(data_wfv, shuffle=False, test_size=0.2)

    #I split the rest of the data into train and cross-validation (cv).
    #cv is the data used as "test" in this program, I only want to discover the best combinations of
    #data-transformer-model-parameters
    data_train, data_cv = train_test_split(data_no_test, shuffle=False, test_size=0.2)
    
    #I separate the variable I want to predict (if bitcoin price will go up or down the following day)
    X_train=data_train.drop(labels=['subida', 'varPSig'], axis=1)
    y_train=data_train['subida']
    real_train=data_train['varPSig']

    X_test=data_cv.drop(labels=['subida', 'varPSig'], axis=1)
    y_test=data_cv['subida']
    real_test=data_cv['varPSig']
    
    #litado de scalers

    number_columns = X_train.select_dtypes('number').columns

    tSS=[
        #('dia', 
        #OneHotEncoder(handle_unknown='ignore'),
        #['dia']),
        ('scaler', StandardScaler(),number_columns)
        ]
    tMM=[
        ('scaler', MinMaxScaler(),number_columns)
        ]
    #tPTbc=[
    #    ('scaler', PowerTransformer(method='box-cox', standardize=False),number_columns)
    #    ]
    tPTyj=[
        ('scaler', PowerTransformer(method='yeo-johnson', standardize=False),number_columns)
        ]
    tQT=[
        ('scaler', QuantileTransformer(output_distribution='normal', random_state=0),number_columns)
        ]
    tMA=[
        ('scaler', MaxAbsScaler(),number_columns)
        ]

    ct_list=[]
    ct_list.append(['StandardScaler', ColumnTransformer(transformers=tSS, remainder='passthrough')])
    ct_list.append(['MinMaxScaler', ColumnTransformer(transformers=tMM, remainder='passthrough')])
    #ct_list.append(['PowerTransformer_box-cox', ColumnTransformer(transformers=tPTbc, remainder='passthrough')])
    ct_list.append(['PowerTransformer_yeo-johnson', ColumnTransformer(transformers=tPTyj, remainder='passthrough')])
    ct_list.append(['QuantileTransformer', ColumnTransformer(transformers=tQT, remainder='passthrough')])
    ct_list.append(['MaxAbsScaler', ColumnTransformer(transformers=tMA, remainder='passthrough')])
    
    #bucle de scalers
    for c in ct_list:
        #bucle de modelos
        for m in modelos:
            print(d[0] + ' ' + c[0] + ' ' + m[0] + ' ' + str(len(y_test)) + ' ' + str(datetime.datetime.now()))
            conf_mat, f1, acc, res_naive, res_mod, res_comis = \
                walk_forward_validation (m[1], X_train, y_train, X_test, y_test, real_test, c[1])
            resultados.append([d[0], c[0], m[0], conf_mat, f1, acc, res_naive, res_mod, res_comis])
            print(resultados[-1])

data_for_use_w_vol StandardScaler KNeighbors_1 336 2022-01-12 00:00:20.711453
['data_for_use_w_vol', 'StandardScaler', 'KNeighbors_1', array([[ 72, 101],
       [ 61, 102]]), 0.5573770491803279, 0.5178571428571429, 1.086943915437065, 1.3287707409688183, 1.052933112629567]
data_for_use_w_vol StandardScaler KNeighbors_2 336 2022-01-12 00:00:24.895216
['data_for_use_w_vol', 'StandardScaler', 'KNeighbors_2', array([[113,  60],
       [104,  59]]), 0.41843971631205673, 0.5119047619047619, 1.086943915437065, 1.0942556730049209, 0.8855163105363277]
data_for_use_w_vol StandardScaler KNeighbors_3 336 2022-01-12 00:00:28.873633
['data_for_use_w_vol', 'StandardScaler', 'KNeighbors_3', array([[ 59, 114],
       [ 54, 109]]), 0.5647668393782385, 0.5, 1.086943915437065, 1.301205765569607, 1.0310903112705547]
data_for_use_w_vol StandardScaler KNeighbors_4 336 2022-01-12 00:00:32.829533
['data_for_use_w_vol', 'StandardScaler', 'KNeighbors_4', array([[82, 91],
       [76, 87]]), 0.5102639296187683, 0.5

['data_for_use_w_vol', 'StandardScaler', 'DecisionTree_11', array([[ 59, 114],
       [ 48, 115]]), 0.5867346938775511, 0.5178571428571429, 1.086943915437065, 1.191404367788142, 0.978715021508478]
data_for_use_w_vol StandardScaler DecisionTree_12 336 2022-01-12 00:04:41.246458
['data_for_use_w_vol', 'StandardScaler', 'DecisionTree_12', array([[ 65, 108],
       [ 55, 108]]), 0.5699208443271767, 0.5148809523809523, 1.086943915437065, 0.7974099540485737, 0.6337766597462514]
data_for_use_w_vol StandardScaler DecisionTree_13 336 2022-01-12 00:05:08.272965
['data_for_use_w_vol', 'StandardScaler', 'DecisionTree_13', array([[ 63, 110],
       [ 60, 103]]), 0.5478723404255319, 0.49404761904761907, 1.086943915437065, 0.7794161477540594, 0.6157667929945582]
data_for_use_w_vol StandardScaler DecisionTree_14 336 2022-01-12 00:05:36.234058
['data_for_use_w_vol', 'StandardScaler', 'DecisionTree_14', array([[ 65, 108],
       [ 52, 111]]), 0.581151832460733, 0.5238095238095238, 1.086943915437065, 1.1

['data_for_use_w_vol', 'StandardScaler', 'RandomForest_3', array([[ 33, 140],
       [ 38, 125]]), 0.5841121495327103, 0.47023809523809523, 1.086943915437065, 0.7748349242934891, 0.6608524789624856]
data_for_use_w_vol StandardScaler RandomForest_4 336 2022-01-12 00:27:27.438164
['data_for_use_w_vol', 'StandardScaler', 'RandomForest_4', array([[ 36, 137],
       [ 42, 121]]), 0.5748218527315915, 0.46726190476190477, 1.086943915437065, 0.7262059657570823, 0.6156691961747758]
data_for_use_w_vol StandardScaler RandomForest_5 336 2022-01-12 00:30:46.513237
['data_for_use_w_vol', 'StandardScaler', 'RandomForest_5', array([[ 29, 144],
       [ 34, 129]]), 0.591743119266055, 0.47023809523809523, 1.086943915437065, 0.6915737570132802, 0.598760508704072]
data_for_use_w_vol StandardScaler RandomForest_6 336 2022-01-12 00:34:01.511523
['data_for_use_w_vol', 'StandardScaler', 'RandomForest_6', array([[ 24, 149],
       [ 39, 124]]), 0.5688073394495413, 0.44047619047619047, 1.086943915437065, 0.7232

['data_for_use_w_vol', 'MinMaxScaler', 'KNeighbors_13', array([[ 40, 133],
       [ 36, 127]]), 0.6004728132387707, 0.49702380952380953, 1.086943915437065, 1.1854021936540746, 1.0418367634184458]
data_for_use_w_vol MinMaxScaler KNeighbors_14 336 2022-01-12 01:23:25.646015
['data_for_use_w_vol', 'MinMaxScaler', 'KNeighbors_14', array([[ 51, 122],
       [ 49, 114]]), 0.5714285714285714, 0.49107142857142855, 1.086943915437065, 1.171764536410813, 0.9859804609566709]
data_for_use_w_vol MinMaxScaler KNeighbors_15 336 2022-01-12 01:23:29.254551
['data_for_use_w_vol', 'MinMaxScaler', 'KNeighbors_15', array([[ 35, 138],
       [ 30, 133]]), 0.6129032258064516, 0.5, 1.086943915437065, 1.470756068573889, 1.3200841965629542]
data_for_use_w_vol MinMaxScaler KNeighbors_16 336 2022-01-12 01:23:32.872948
['data_for_use_w_vol', 'MinMaxScaler', 'KNeighbors_16', array([[ 50, 123],
       [ 46, 117]]), 0.5806451612903225, 0.49702380952380953, 1.086943915437065, 1.1028989471021116, 0.940656493900465]
data

['data_for_use_w_vol', 'MinMaxScaler', 'AdaBoost_4', array([[ 17, 156],
       [ 18, 145]]), 0.625, 0.48214285714285715, 1.086943915437065, 0.9272583572722775, 0.8666901694869803]
data_for_use_w_vol MinMaxScaler AdaBoost_5 336 2022-01-12 01:32:04.953119
['data_for_use_w_vol', 'MinMaxScaler', 'AdaBoost_5', array([[ 34, 139],
       [ 36, 127]]), 0.5920745920745921, 0.4791666666666667, 1.086943915437065, 0.7147912782864929, 0.6235245283406577]
data_for_use_w_vol MinMaxScaler AdaBoost_6 336 2022-01-12 01:32:22.505504
['data_for_use_w_vol', 'MinMaxScaler', 'AdaBoost_6', array([[ 41, 132],
       [ 35, 128]]), 0.6052009456264776, 0.5029761904761905, 1.086943915437065, 0.7439253122755661, 0.6528469160929506]
data_for_use_w_vol MinMaxScaler AdaBoost_7 336 2022-01-12 01:32:43.041233
['data_for_use_w_vol', 'MinMaxScaler', 'AdaBoost_7', array([[ 49, 124],
       [ 45, 118]]), 0.5827160493827162, 0.49702380952380953, 1.086943915437065, 0.6932167383448414, 0.6028919385566965]
data_for_use_w_vol Mi

...

['data_for_use_bone_deep', 'QuantileTransformer', 'KNeighbors_17', array([[ 29, 144],
       [ 28, 135]]), 0.6108597285067873, 0.4880952380952381, 1.086943915437065, 0.8552411980453183, 0.738243098230438]
data_for_use_bone_deep QuantileTransformer KNeighbors_18 336 2022-01-15 23:07:21.757694
['data_for_use_bone_deep', 'QuantileTransformer', 'KNeighbors_18', array([[ 42, 131],
       [ 44, 119]]), 0.5762711864406779, 0.4791666666666667, 1.086943915437065, 0.6576285478794307, 0.5426671295444558]
data_for_use_bone_deep QuantileTransformer KNeighbors_19 336 2022-01-15 23:07:47.146770
['data_for_use_bone_deep', 'QuantileTransformer', 'KNeighbors_19', array([[ 27, 146],
       [ 31, 132]]), 0.5986394557823129, 0.4732142857142857, 1.086943915437065, 0.6793700246887988, 0.5881945887272172]
data_for_use_bone_deep QuantileTransformer KNeighbors_20 336 2022-01-15 23:08:12.567904
['data_for_use_bone_deep', 'QuantileTransformer', 'KNeighbors_20', array([[ 43, 130],
       [ 41, 122]]), 0.5879518072

['data_for_use_bone_deep', 'QuantileTransformer', 'AdaBoost_5', array([[ 35, 138],
       [ 41, 122]]), 0.5768321513002365, 0.46726190476190477, 1.086943915437065, 0.8407439645677129, 0.7400305367557579]
data_for_use_bone_deep QuantileTransformer AdaBoost_6 336 2022-01-15 23:21:42.762195
['data_for_use_bone_deep', 'QuantileTransformer', 'AdaBoost_6', array([[ 33, 140],
       [ 42, 121]]), 0.5707547169811321, 0.4583333333333333, 1.086943915437065, 0.8720474253057532, 0.7607017266046135]
data_for_use_bone_deep QuantileTransformer AdaBoost_7 336 2022-01-15 23:22:16.025966
['data_for_use_bone_deep', 'QuantileTransformer', 'AdaBoost_7', array([[ 34, 139],
       [ 44, 119]]), 0.5653206650831354, 0.45535714285714285, 1.086943915437065, 0.9269564693206744, 0.8013496523056464]
data_for_use_bone_deep QuantileTransformer AdaBoost_8 336 2022-01-15 23:22:50.688003
['data_for_use_bone_deep', 'QuantileTransformer', 'AdaBoost_8', array([[ 32, 141],
       [ 45, 118]]), 0.5592417061611374, 0.44642857

['data_for_use_bone_deep', 'QuantileTransformer', 'RandomForest_15', array([[ 30, 143],
       [ 42, 121]]), 0.5667447306791569, 0.4494047619047619, 1.086943915437065, 0.6381911374513447, 0.5492343482827972]
data_for_use_bone_deep QuantileTransformer RandomForest_16 336 2022-01-16 00:08:37.437509
['data_for_use_bone_deep', 'QuantileTransformer', 'RandomForest_16', array([[ 31, 142],
       [ 46, 117]]), 0.5545023696682464, 0.44047619047619047, 1.086943915437065, 0.8342150223715028, 0.7072380515697744]
data_for_use_bone_deep QuantileTransformer RandomForest_17 336 2022-01-16 00:11:07.990003
['data_for_use_bone_deep', 'QuantileTransformer', 'RandomForest_17', array([[ 33, 140],
       [ 42, 121]]), 0.5707547169811321, 0.4583333333333333, 1.086943915437065, 0.623154787100015, 0.5330833682758818]
data_for_use_bone_deep QuantileTransformer RandomForest_18 336 2022-01-16 00:13:37.380471
['data_for_use_bone_deep', 'QuantileTransformer', 'RandomForest_18', array([[ 40, 133],
       [ 41, 122]]

['data_for_use_bone_deep', 'MaxAbsScaler', 'DecisionTree_4', array([[ 40, 133],
       [ 45, 118]]), 0.570048309178744, 0.47023809523809523, 1.086943915437065, 0.8339073558607458, 0.6974900881541345]
data_for_use_bone_deep MaxAbsScaler DecisionTree_5 336 2022-01-16 00:22:08.757644
['data_for_use_bone_deep', 'MaxAbsScaler', 'DecisionTree_5', array([[ 43, 130],
       [ 55, 108]]), 0.5386533665835411, 0.4494047619047619, 1.086943915437065, 0.6438290649596018, 0.5273072991802255]
data_for_use_bone_deep MaxAbsScaler DecisionTree_6 336 2022-01-16 00:22:15.249285
['data_for_use_bone_deep', 'MaxAbsScaler', 'DecisionTree_6', array([[ 48, 125],
       [ 60, 103]]), 0.5268542199488492, 0.4494047619047619, 1.086943915437065, 0.5626252959270348, 0.4539344577964232]
data_for_use_bone_deep MaxAbsScaler DecisionTree_7 336 2022-01-16 00:22:22.523831
['data_for_use_bone_deep', 'MaxAbsScaler', 'DecisionTree_7', array([[ 49, 124],
       [ 58, 105]]), 0.5357142857142858, 0.4583333333333333, 1.08694391543

['data_for_use_bone_deep', 'MaxAbsScaler', 'AdaBoost_14', array([[ 32, 141],
       [ 49, 114]]), 0.5454545454545454, 0.43452380952380953, 1.086943915437065, 0.5218952283386273, 0.43651941221384144]
data_for_use_bone_deep MaxAbsScaler AdaBoost_15 336 2022-01-16 00:27:44.664183
['data_for_use_bone_deep', 'MaxAbsScaler', 'AdaBoost_15', array([[ 37, 136],
       [ 48, 115]]), 0.5555555555555556, 0.4523809523809524, 1.086943915437065, 0.5548907904107728, 0.45857703933933575]
data_for_use_bone_deep MaxAbsScaler AdaBoost_16 336 2022-01-16 00:28:07.594652
['data_for_use_bone_deep', 'MaxAbsScaler', 'AdaBoost_16', array([[ 37, 136],
       [ 49, 114]]), 0.5520581113801453, 0.4494047619047619, 1.086943915437065, 0.603108771775552, 0.4969315521312528]
data_for_use_bone_deep MaxAbsScaler AdaBoost_17 336 2022-01-16 00:28:31.939339
['data_for_use_bone_deep', 'MaxAbsScaler', 'AdaBoost_17', array([[ 37, 136],
       [ 46, 117]]), 0.5625, 0.4583333333333333, 1.086943915437065, 0.7642118564695541, 0.627

In [19]:
resultados

[['data_for_use_w_vol',
  'StandardScaler',
  'KNeighbors_1',
  array([[ 72, 101],
         [ 61, 102]]),
  0.5573770491803279,
  0.5178571428571429,
  1.086943915437065,
  1.3287707409688183,
  1.052933112629567],
 ['data_for_use_w_vol',
  'StandardScaler',
  'KNeighbors_2',
  array([[113,  60],
         [104,  59]]),
  0.41843971631205673,
  0.5119047619047619,
  1.086943915437065,
  1.0942556730049209,
  0.8855163105363277],
 ['data_for_use_w_vol',
  'StandardScaler',
  'KNeighbors_3',
  array([[ 59, 114],
         [ 54, 109]]),
  0.5647668393782385,
  0.5,
  1.086943915437065,
  1.301205765569607,
  1.0310903112705547],
 ['data_for_use_w_vol',
  'StandardScaler',
  'KNeighbors_4',
  array([[82, 91],
         [76, 87]]),
  0.5102639296187683,
  0.5029761904761905,
  1.086943915437065,
  1.0141416037933806,
  0.7892710200498854],
 ['data_for_use_w_vol',
  'StandardScaler',
  'KNeighbors_5',
  array([[ 43, 130],
         [ 51, 112]]),
  0.5530864197530865,
  0.46130952380952384,
  1.0

In [20]:
resulados_df = pd.DataFrame(resultados)

In [21]:
resulados_df.to_csv('prueba_bucle_resultados_completo.csv')

In [5]:
resulados_df = pd.read_csv('prueba_bucle_resultados_completo.csv', index_col=0)

In [6]:
#This is how much better is the model compared to the naive model
resulados_df['model_boost']=resulados_df['8']-resulados_df['6']

In [7]:
#This is the negative accuracy (True negatives/(True negatives+False negatives))
resulados_df['neg_acc'] = resulados_df['3'].str.split('\W+', expand=True)[1].str.strip().astype(int) \
    /(resulados_df['3'].str.split('\W+', expand=True)[1].str.strip().astype(int) + \
      resulados_df['3'].str.split('\W+', expand=True)[3].str.strip().astype(int))

In [8]:
resulados_df=resulados_df[resulados_df['model_boost']>0]

In [9]:
resulados_df.sort_values(by='4', axis=0, ascending=False).head(20)

,0,1,2,3,4,5,6,7,8,model_boost,neg_acc
1916,data_for_use_basic,MaxAbsScaler,DecisionTree_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000
1936,data_for_use_basic,MaxAbsScaler,AdaBoost_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000
1778,data_for_use_basic,PowerTransformer_yeo-johnson,AdaBoost_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000
1837,data_for_use_basic,QuantileTransformer,DecisionTree_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000
1679,data_for_use_basic,MinMaxScaler,DecisionTree_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000
1600,data_for_use_basic,StandardScaler,DecisionTree_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000
1699,data_for_use_basic,MinMaxScaler,AdaBoost_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000
1620,data_for_use_basic,StandardScaler,AdaBoost_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000
1758,data_for_use_basic,PowerTransformer_yeo-johnson,DecisionTree_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000
1857,data_for_use_basic,QuantileTransformer,AdaBoost_1,[[ 15 158]\n [ 10 153]],0.645570,0.500000,1.086944,1.125643,1.108872,0.021928,0.600000


In [10]:
resulados_df.sort_values(by='5', axis=0, ascending=False).head(12)

,0,1,2,3,4,5,6,7,8,model_boost,neg_acc
1766,data_for_use_basic,PowerTransformer_yeo-johnson,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
1687,data_for_use_basic,MinMaxScaler,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
1608,data_for_use_basic,StandardScaler,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
1924,data_for_use_basic,MaxAbsScaler,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
1845,data_for_use_basic,QuantileTransformer,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
2182,data_for_use_basic_w_gold,PowerTransformer_yeo-johnson,AdaBoost_10,[[ 60 112]\n [ 40 123]],0.618090,0.546269,1.205127,1.565161,1.400607,0.195480,0.600000
2933,data_for_use_basic_w_all,PowerTransformer_yeo-johnson,KNeighbors_11,[[90 82]\n [70 93]],0.550296,0.546269,1.205127,2.348387,2.107807,0.902680,0.562500
2145,data_for_use_basic_w_gold,PowerTransformer_yeo-johnson,KNeighbors_13,[[89 83]\n [69 94]],0.552941,0.546269,1.205127,1.889287,1.716226,0.511099,0.563291
2261,data_for_use_basic_w_gold,QuantileTransformer,AdaBoost_10,[[ 60 112]\n [ 40 123]],0.618090,0.546269,1.205127,1.565161,1.400607,0.195480,0.600000
2103,data_for_use_basic_w_gold,MinMaxScaler,AdaBoost_10,[[ 60 112]\n [ 40 123]],0.618090,0.546269,1.205127,1.565161,1.400607,0.195480,0.600000


In [11]:
resulados_df.sort_values(by='model_boost', axis=0, ascending=False).head(20)

,0,1,2,3,4,5,6,7,8,model_boost,neg_acc
2931,data_for_use_basic_w_all,PowerTransformer_yeo-johnson,KNeighbors_9,[[89 83]\n [75 88]],0.526946,0.528358,1.205127,2.522635,2.223782,1.018656,0.542683
2933,data_for_use_basic_w_all,PowerTransformer_yeo-johnson,KNeighbors_11,[[90 82]\n [70 93]],0.550296,0.546269,1.205127,2.348387,2.107807,0.902680,0.562500
2142,data_for_use_basic_w_gold,PowerTransformer_yeo-johnson,KNeighbors_10,[[113 59]\n [102 61]],0.431095,0.519403,1.205127,2.253073,2.010150,0.805023,0.525581
2536,data_for_use_basic_w_nasdaq,PowerTransformer_yeo-johnson,KNeighbors_9,[[92 80]\n [75 88]],0.531722,0.537313,1.205127,2.253257,1.998279,0.793153,0.550898
2141,data_for_use_basic_w_gold,PowerTransformer_yeo-johnson,KNeighbors_9,[[92 80]\n [79 84]],0.513761,0.525373,1.205127,2.251310,1.966806,0.761679,0.538012
2932,data_for_use_basic_w_all,PowerTransformer_yeo-johnson,KNeighbors_10,[[115 57]\n [ 98 65]],0.456140,0.537313,1.205127,2.184041,1.954420,0.749293,0.539906
2143,data_for_use_basic_w_gold,PowerTransformer_yeo-johnson,KNeighbors_11,[[88 84]\n [71 92]],0.542773,0.537313,1.205127,2.192000,1.943955,0.738828,0.553459
1056,data_for_use_w_nasdaq,QuantileTransformer,DecisionTree_10,[[ 76 96]\n [ 62 101]],0.561111,0.528358,1.205127,2.378800,1.922134,0.717007,0.550725
977,data_for_use_w_nasdaq,PowerTransformer_yeo-johnson,DecisionTree_10,[[ 75 97]\n [ 62 101]],0.559557,0.525373,1.205127,2.369083,1.914283,0.709156,0.547445
819,data_for_use_w_nasdaq,StandardScaler,DecisionTree_10,[[ 75 97]\n [ 62 101]],0.559557,0.525373,1.205127,2.369083,1.914283,0.709156,0.547445


In [12]:
resulados_df.sort_values(by='neg_acc', axis=0, ascending=False).head(20)

,0,1,2,3,4,5,6,7,8,model_boost,neg_acc
1608,data_for_use_basic,StandardScaler,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
1766,data_for_use_basic,PowerTransformer_yeo-johnson,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
1845,data_for_use_basic,QuantileTransformer,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
1687,data_for_use_basic,MinMaxScaler,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
1924,data_for_use_basic,MaxAbsScaler,DecisionTree_9,[[ 58 115]\n [ 36 127]],0.627160,0.550595,1.086944,1.471163,1.239769,0.152825,0.617021
614,data_for_use_w_gold,PowerTransformer_yeo-johnson,RandomForest_3,[[ 48 124]\n [ 30 133]],0.633333,0.540299,1.205127,1.807892,1.537319,0.332192,0.615385
2023,data_for_use_basic_w_gold,StandardScaler,AdaBoost_9,[[ 55 117]\n [ 36 127]],0.624079,0.543284,1.205127,1.511357,1.368799,0.163672,0.604396
2181,data_for_use_basic_w_gold,PowerTransformer_yeo-johnson,AdaBoost_9,[[ 55 117]\n [ 36 127]],0.624079,0.543284,1.205127,1.511357,1.368799,0.163672,0.604396
2339,data_for_use_basic_w_gold,MaxAbsScaler,AdaBoost_9,[[ 55 117]\n [ 36 127]],0.624079,0.543284,1.205127,1.511357,1.368799,0.163672,0.604396
2260,data_for_use_basic_w_gold,QuantileTransformer,AdaBoost_9,[[ 55 117]\n [ 36 127]],0.624079,0.543284,1.205127,1.511357,1.368799,0.163672,0.604396


In [13]:
resulados_df[(resulados_df['model_boost']>0.2)&(resulados_df['5']>0.53)&(resulados_df['4']>0.6)]

,0,1,2,3,4,5,6,7,8,model_boost,neg_acc
456,data_for_use_w_gold,StandardScaler,RandomForest_3,[[ 46 126]\n [ 31 132]],0.627078,0.531343,1.205127,1.641371,1.408348,0.203221,0.597403
535,data_for_use_w_gold,MinMaxScaler,RandomForest_3,[[ 46 126]\n [ 31 132]],0.627078,0.531343,1.205127,1.641371,1.408348,0.203221,0.597403
614,data_for_use_w_gold,PowerTransformer_yeo-johnson,RandomForest_3,[[ 48 124]\n [ 30 133]],0.633333,0.540299,1.205127,1.807892,1.537319,0.332192,0.615385
772,data_for_use_w_gold,MaxAbsScaler,RandomForest_3,[[ 46 126]\n [ 31 132]],0.627078,0.531343,1.205127,1.641371,1.408348,0.203221,0.597403
1610,data_for_use_basic,StandardScaler,DecisionTree_11,[[ 66 107]\n [ 46 117]],0.604651,0.544643,1.086944,1.885159,1.498313,0.411370,0.589286
1689,data_for_use_basic,MinMaxScaler,DecisionTree_11,[[ 66 107]\n [ 46 117]],0.604651,0.544643,1.086944,1.885159,1.498313,0.411370,0.589286
1768,data_for_use_basic,PowerTransformer_yeo-johnson,DecisionTree_11,[[ 66 107]\n [ 46 117]],0.604651,0.544643,1.086944,1.885159,1.498313,0.411370,0.589286
1926,data_for_use_basic,MaxAbsScaler,DecisionTree_11,[[ 66 107]\n [ 46 117]],0.604651,0.544643,1.086944,1.885159,1.498313,0.411370,0.589286
2394,data_for_use_basic_w_nasdaq,StandardScaler,DecisionTree_5,[[ 51 121]\n [ 35 128]],0.621359,0.534328,1.205127,1.650008,1.437171,0.232044,0.593023
2473,data_for_use_basic_w_nasdaq,MinMaxScaler,DecisionTree_5,[[ 51 121]\n [ 35 128]],0.621359,0.534328,1.205127,1.650008,1.437171,0.232044,0.593023


In [14]:
def find_in_list(list_find, index_find, index_ret, value_find):
    val_ret=''
    for i in range(len(list_find)):
        if list_find[i][index_find]==value_find:
            val_ret=list_find[i][index_ret]
            break
            
    return val_ret
    

In [15]:
back_test_models=[]
for r in range(len(resulados_df[resulados_df['1']=='PowerTransformer_yeo-johnson'])):
    back_test_models.append([resulados_df[resulados_df['1']=='PowerTransformer_yeo-johnson']['0'].iloc[r], \
                            'PowerTransformer_yeo-johnson', \
                            resulados_df[resulados_df['1']=='PowerTransformer_yeo-johnson']['2'].iloc[r]])


In [16]:
dates=[]
dates.append(['2016-01-01','2017-12-31','2016-17'])
dates.append(['2018-01-01','2018-03-31','2018Q1'])
dates.append(['2018-04-01','2018-06-30','2018Q2'])
dates.append(['2018-07-01','2018-09-30','2018Q3'])
dates.append(['2018-10-01','2018-12-31','2018Q4'])
dates.append(['2019-01-01','2019-03-31','2019Q1'])
dates.append(['2019-04-01','2019-06-30','2019Q2'])
dates.append(['2019-07-01','2019-09-30','2019Q3'])
dates.append(['2019-10-01','2019-12-31','2019Q4'])

In [158]:
for b in range(len(back_test_models)):

    data_wfv = find_in_list(data,0,1,back_test_models[b][0])
    model = find_in_list(modelos,0,1,back_test_models[b][2])
    
    #I split the data into test and the rest (test data is not meant to be used here, this program is only for CV)
    data_no_test, data_test = train_test_split(data_wfv, shuffle=False, test_size=0.2)

    #I split the rest of the data into train and cross-validation (cv).
    #cv is the data used as "test" in this program, I only want to discover the best combinations of
    #data-transformer-model-parameters
    data_train_backtest, data_cv = train_test_split(data_no_test, shuffle=False, test_size=0.2)
    
    res_naive_acum=1
    res_comis_acum=1

    for i in range(len(dates)):
        data_train=data_train_backtest[dates[0][0]:dates[i][1]].copy()
        data_cv=data_train_backtest[dates[i+1][0]:dates[i+1][1]].copy()
    
        if data_cv.size==0:
            break
    
    
        #I separate the variable I want to predict (if bitcoin price will go up or down the following day)
        X_train=data_train.drop(labels=['subida', 'varPSig'], axis=1)
        y_train=data_train['subida']
        real_train=data_train['varPSig']

        X_test=data_cv.drop(labels=['subida', 'varPSig'], axis=1)
        y_test=data_cv['subida']
        real_test=data_cv['varPSig']
    
        number_columns = X_train.select_dtypes('number').columns

        transf=[
            ('scaler', PowerTransformer(method='yeo-johnson', standardize=False),number_columns)
            #('scaler', StandardScaler(),number_columns)
            #('scaler', QuantileTransformer(output_distribution='normal', random_state=0),number_columns)
            ]

        coltr=ColumnTransformer(transformers=transf, remainder='passthrough')

    
        conf_mat, f1, acc, res_naive, res_mod, res_comis = \
                walk_forward_validation (model, X_train, y_train, X_test, y_test, real_test, coltr)
        #print([dates[i+1][2], conf_mat, f1, acc, res_naive, res_comis])
        res_naive_acum=res_naive_acum*res_naive
        res_comis_acum=res_comis_acum*res_comis
        #print([res_naive_acum, res_comis_acum])
    
    if res_comis_acum>res_naive_acum:
        print([back_test_models[b][0], back_test_models[b][2]], res_naive_acum, res_comis_acum)


['data_for_use_w_vol', 'KNeighbors_16'] 0.7576048234094187 2.239307066450511
['data_for_use_basic', 'KNeighbors_2'] 0.7576048234094187 1.0309726365702885
['data_for_use_basic', 'KNeighbors_3'] 0.7576048234094187 1.0336090899302615
['data_for_use_basic', 'KNeighbors_4'] 0.7576048234094187 1.2549176347590913
['data_for_use_basic', 'KNeighbors_5'] 0.7576048234094187 0.9632987574897869
['data_for_use_basic', 'KNeighbors_7'] 0.7576048234094187 1.180988061540778
['data_for_use_basic', 'KNeighbors_8'] 0.7576048234094187 1.1331433000295694
['data_for_use_basic', 'KNeighbors_9'] 0.7576048234094187 1.461708175653119
['data_for_use_basic', 'KNeighbors_10'] 0.7576048234094187 1.7551144626749382
['data_for_use_basic', 'KNeighbors_11'] 0.7576048234094187 1.3538809435435362
['data_for_use_basic', 'KNeighbors_12'] 0.7576048234094187 2.091998024451233
['data_for_use_basic', 'KNeighbors_13'] 0.7576048234094187 1.4891121802581861
['data_for_use_basic', 'KNeighbors_14'] 0.7576048234094187 1.55901297521138

In [41]:
test_models=[]
test_models.append(['data_for_use_basic','PowerTransformer_yeo-johnson', 'KNeighbors_13'])
test_models

[['data_for_use_basic', 'PowerTransformer_yeo-johnson', 'KNeighbors_13']]

In [42]:
data_wfv = find_in_list(data,0,1,test_models[0][0])
model = find_in_list(modelos,0,1,test_models[0][2])
    
#I split the data into test and the rest (test data is not meant to be used here, this program is only for CV)
data_no_test, data_test = train_test_split(data_wfv, shuffle=False, test_size=0.2)

model

KNeighborsClassifier(n_neighbors=13)

In [43]:
#I separate the variable I want to predict (if bitcoin price will go up or down the following day)
X_train=data_no_test.drop(labels=['subida', 'varPSig'], axis=1).copy()
y_train=data_no_test['subida'].copy()
real_train=data_no_test['varPSig'].copy()

X_test=data_test.drop(labels=['subida', 'varPSig'], axis=1).copy()
y_test=data_test['subida'].copy()
real_test=data_test['varPSig'].copy()
    
number_columns = X_train.select_dtypes('number').columns

transf=[
    ('scaler', PowerTransformer(method='yeo-johnson', standardize=False),number_columns)
    #('scaler', StandardScaler(),number_columns)
    #('scaler', QuantileTransformer(output_distribution='normal', random_state=0),number_columns)
    ]

coltr=ColumnTransformer(transformers=transf, remainder='passthrough')

    
conf_mat, f1, acc, res_naive, res_mod, res_comis = \
            walk_forward_validation (model, X_train, y_train, X_test, y_test, real_test, coltr)
print([conf_mat, f1, acc, res_naive, res_mod, res_comis])

[array([[ 59, 129],
       [ 75, 156]]), 0.6046511627906976, 0.513126491646778, 3.8420060121748145, 3.055783230292584, 2.6816652494895825]
